In [65]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [66]:
tweets = pd.read_csv('covid19_tweets.csv')


In [67]:
## If you comment out the commands below, you can see the format of the dataframe. 
## The first column is an index, then you have user_name, user_location, user_description,
## user_created, user_followers, user_friends, user_favourites, user_verified (True/False),
## date, text, hashtags (presented as a list of strings), 
## source (Twitter Web App, for android, etc), and is_retweet (True/False)

##print(len(tweets))
##print(tweets.head(30))


166656
                               user_name  \
0                                 ᏉᎥ☻լꂅϮ   
1                          Tom Basile 🇺🇸   
2                        Time4fisticuffs   
3                            ethel mertz   
4                               DIPR-J&K   
5                       🎹 Franz Schubert   
6                           hr bartender   
7                         Derbyshire LPC   
8                      Prathamesh Bendre   
9      Member of Christ 🇨🇳🇺🇸🇮🇳🇮🇩🇧🇷🇳🇬🇧🇩🇷🇺   
10                Voice Of CBSE Students   
11                           Creativegms   
12                            SEXXYLYPPS   
13    Africa Youth Advisory Board on DRR   
14                        DailyaddaaNews   
15                         Dimapur 24/7.   
16                        ChennaiCityNow   
17                  marc goovaerts🇪🇺🏳️‍🌈   
18                            Dorian Aur   
19                       Coronavirus Law   
20                     The Voice of GenX   
21                     AP

In [143]:
## Our first goal is to extract a subset of the tweet texts, labeled according to political
## leaning. It's hard to extract someone's political stance just from the information
## available in this table, but we will make a guess that a user_description that
## uses a lot of right-wing adjectives (conservative, traditional, etc.) is probably
## conservative and a user_description that uses a lot of left-wing adjectives (liberal,
## progressive, etc.) is probably liberal. Of course, many people leave their descriptions blank
## or don't include such terms; we will simply omit their posts. 

## The first thing we do is define a function that will extract the political leaning
## of a tweet's author. We build it out of somewhat simpler functions: 
## first a function that checks if a word is in a user_description, then a function
## that checks if any of a list of words appears in a user_description,
## and finally a function that checks to see if 

def word_in_desc(word,desc):
    return word in str(desc)

def some_word_in_desc(lst,desc):
    state = False
    for word in lst:
        if word_in_desc(word,desc):
            state = True
            break
    return state

def political_leaning(desc, 
                      liberal_words=['liberal','progressive','democratic','leftist'],
                      conservative_words=['conservative','traditional','libertarian','republican']):
    if some_word_in_desc(liberal_words,desc) and not some_word_in_desc(conservative_words,desc):
        return "L"
    elif some_word_in_desc(conservative_words,desc) and not some_word_in_desc(liberal_words,desc):
        return "R"
    else:
        return "I"

        


166656

In [ ]:
## We should check that this works at least somewhat reasonably. 



In [ ]:
## The next function that we want to create will take a tweet and determine its sentiment
## that is, whether it is positive or negative. 

In [72]:
tweet_lst = []
for i in range(len(tweets)):
    tweet_lst.append(tweets.loc[i]['text'])

In [73]:
##Now we use a less naive approach to analyze the sentiment. 

print(tweets.loc[100]['text'])

link_indices = []
no_link_indices = []

for i in range(len(tweet_lst)): 
    if 'https' in tweet_lst[i]:
        link_indices.append(i)
    else:
        no_link_indices.append(i)

# print(len(link_indices))
# print(len(no_link_indices))



Voices from the Belt and Road: COVID-19 rap song alerts to needed precautions when returning to work
#beltandroad… https://t.co/O39NmfUDmc


In [74]:
# for k in no_link_indices:
#     print(tweet_lst[k])

In [148]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *

from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer



## First we are going to use some standard training tweets to create a model. 
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

tknz = TweetTokenizer()

tweet_text_tokenizes = [tknz.tokenize(tweet) for tweet in tweet_texts]


In [149]:
print(tweet_text_tokenizes[10])

['49K', '+', 'Covid', '19', 'cases', 'still', 'no', 'response', 'from', '@cbseindia29', '@HRDMinistry', '@DrRPNishank', '.', 'Please', 'cancel', 'the', 'compartment', 'exa', '…', 'https://t.co/kV2ZKmumu1']


In [76]:
## First we use part-of-speech tagging. 
## We can remove the 

from nltk.tag import pos_tag
from nltk.corpus import twitter_samples

print(pos_tag(tweet_tokens[0]))




[('#FollowFriday', 'JJ'), ('@France_Inte', 'NNP'), ('@PKuchly57', 'NNP'), ('@Milipol_Paris', 'NNP'), ('for', 'IN'), ('being', 'VBG'), ('top', 'JJ'), ('engaged', 'VBN'), ('members', 'NNS'), ('in', 'IN'), ('my', 'PRP$'), ('community', 'NN'), ('this', 'DT'), ('week', 'NN'), (':)', 'NN')]


In [77]:
# print(tweet_tokens[140])
# print(pos_tag(tweet_tokens[140]))

In [78]:
## We need to process the words. The first step is Lemmatization: reducing a word to its root or canonical form.



from nltk.stem.wordnet import WordNetLemmatizer


## If you are missing the two modules, uncomment the code. 
# nltk.download('wordnet')
# nltk.download('averaged_perceptron_tagger')

# The following function uses the pos_tag information to feed the lemmatizer (which can only operate when it 
## knows what part of speech the input word is)
# def lemmatize_sentence(tokens):
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_sentence = []
#     for word, tag in pos_tag(tokens):
#         if tag.startswith('NN'):
#             pos = 'n'
#         elif tag.startswith('VB'):
#             pos = 'v'
#         else:
#             pos = 'a'
#         lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
#     return lemmatized_sentence

# print(lemmatize_sentence(tweet_tokens[200]))

In [150]:
## NOW we need to clean up the lemmatized posts. 

## We do this via a function called remove_noise that begins by removing URLS (identified as a regular expression),
## and twitter handles (identified with a @ symbol). 

## We incorporate the lemmatization from the previous thing into this function. Note that we also have a 
## thing that filters out punctuation, as well as the so-called stop words,
##  which are in the typle of stop_words provided as the second argument)

import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens



KeyboardInterrupt: 

In [80]:
# nltk.download('stopwords')

In [151]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
covid_cleaned_tokens_list = []

for tokens in tweet_text_tokenizes:
    covid_cleaned_tokens_list.append(remove_noise(tokens,stop_words))
    
## Here is an example of what remove noise does: the first list is the list of tokens in the 500th
## positive tweet, the second is what happens to that list after we delete all the 'junk'


print(positive_tweet_tokens[500])
print(positive_cleaned_tokens_list[500])

['Dang', 'that', 'is', 'some', 'rad', '@AbzuGame', '#fanart', '!', ':D', 'https://t.co/bI8k8tb9ht']
['dang', 'rad', '#fanart', ':d']


In [82]:
## The next big step is to actually measure how frequently each word turns up in the set of tweets. 
## This is called measuring the word density. 

# def get_all_words(list_of_list_of_words):
#     for lst in list_of_list_of_words:
#         for word in lst:
#             yield word
            
# ## how this works: lists are iterables, which basically means that for-loops can range over them
# ## a generator is a special kind of iterable which you use to create a list, and you can only iterate through once
# ## this creates a list. (we could also have written it differently: 
# ## return [x for all x in lst for all lst in list_of_list_of_words]

# all_pos_words = get_all_words(positive_cleaned_tokens_list)
# all_neg_words = get_all_words(negative_cleaned_tokens_list)



In [83]:
# from nltk import FreqDist

# freq_dist_pos = FreqDist(all_pos_words)
# freq_dist_neg = FreqDist(all_neg_words)

# # print(freq_dist_neg.most_common(10))

In [90]:
## We are going to use Naïve Bayes for our classification. The NB classifier in NLTK requires the 
## tweets to be converted to dictionary format, with the words as keys and True for each value.



def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)



In [91]:
## Now lets label the training tweets as either Positive or Negative
import random
## we import the random module because we will randomly shuffle the dataset, 
## after it's been labeled, into a training chunk and a testing chunk

positive_dataset = [(tweet_dict,"Positive")
                   for tweet_dict in positive_tokens_for_model]
negative_dataset = [(tweet_dict,"Negative")
                   for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]



In [93]:
## Now we get to some of the actual action -- building and training the model. 

from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(20))



Accuracy is: 0.9963333333333333
Most Informative Features
                      :( = True           Negati : Positi =   2074.4 : 1.0
                      :) = True           Positi : Negati =   1647.2 : 1.0
                follower = True           Positi : Negati =     38.9 : 1.0
                     sad = True           Negati : Positi =     26.2 : 1.0
                 welcome = True           Positi : Negati =     21.4 : 1.0
                     bam = True           Positi : Negati =     21.0 : 1.0
                    glad = True           Positi : Negati =     20.3 : 1.0
                     x15 = True           Negati : Positi =     16.4 : 1.0
                    blog = True           Positi : Negati =     15.0 : 1.0
               community = True           Positi : Negati =     14.3 : 1.0
                   didnt = True           Negati : Positi =     13.7 : 1.0
                  arrive = True           Positi : Negati =     13.5 : 1.0
                     idk = True           

In [122]:
from nltk.tokenize import word_tokenize

def string_to_classification(string):
    tokens = remove_noise(word_tokenize(string))
    print('The classification is: ',classifier.classify(dict([token,True] for token in tokens)))
    probs = classifier.prob_classify(dict([token,True] for token in tokens))
    print('Negative probability ',probs.prob('Negative'))
    print('Positive probability ',probs.prob('Positive'))

In [123]:
string_to_classification('people aren’t aware of this but HW assassinated 25 percent of Americans after the convention and that explains this shift')

The classification is:  Negative
Negative probability  0.8439478115392183
Positive probability  0.15605218846078261


In [141]:
## Now lets start actually processing the Covid tweets. 

## Our question is: if we can tag a Covid-related tweet as coming from a 
## politically conservative or politically liberal account, is that 
## associated with the sentiment as predicted by our Naive Bayes classifier? 





[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]


[0, 1, 2, 3, 4]